In [1]:
import os
import urllib
import json
import pandas as pd
from pandas import json_normalize
from datetime import datetime
from datetime import date
import time

In [2]:
# GitHub API unauthenticated requests rate limit = 10 requests per minute.
orgs = ["nhsx","111Online","NHSDigital","nhsconnect","nhsengland"]

In [3]:
# last modified datex
data_updated = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(data_updated)

31/03/2021 16:52:29


In [4]:
df = pd.DataFrame()
for org in orgs:
    data = [1]
    page = 1
    while bool(data) is True:
        url = "https://api.github.com/orgs/"+str(org)+"/repos?page="+str(page)+"&per_page=100"
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        flat_data = json_normalize(data)
        df = df.append(flat_data)
        page = page + 1
        time.sleep(10) # Sleep to avoid rate limit
df.columns = df.columns.str.replace('.', '_')

In [11]:
# Group the data frame by month and item and extract a number of stats from each group
df_group = df.groupby(
    ['owner_login']
).agg(
    {
        # find the number of open repos
        'name': "count",
        'size': "sum",
        'stargazers_count': "sum",
        'forks_count': "sum",
        'open_issues_count': "sum",
        'license_name': lambda x:x.value_counts().index[0],
        'language': lambda x:x.value_counts().index[0]
    }
)
df_group = df_group.reset_index()
df_group.columns = ["Org","Open Repos","Total Size","Stargazers","Forks","Open Issues","Top License","Top Language"]
df_group

,Org,Open Repos,Total Size,Stargazers,Forks,Open Issues,Top License,Top Language
0,111Online,13,522097,10,0,26,Apache License 2.0,C#
1,NHSDigital,83,495092,91,74,372,MIT License,Python
2,nhsconnect,178,3193094,243,198,405,Apache License 2.0,CSS
3,nhsengland,23,36298,14,38,80,MIT License,Python
4,nhsx,32,428854,197,85,51,MIT License,Python


In [12]:
df_html = df_group.head(10).to_html(classes='summary')

In [13]:
# Write HTML String to file.html
with open("_includes/table.html", "w") as file:
    file.write(df_html)

In [ ]:
import plotly 
# Use init_notebook_mode() to view the plots in jupyter notebook
plotly.offline.init_notebook_mode()
from plotly.graph_objs import Scatter,Layout,Bar

trace1 = Bar(x=data_canada['year'],y=data_canada['pop'])
    
# Create chart 
plotly.offline.iplot(
    {
        "data": [trace1],
        "layout": Layout(
            title="<b>Simple Bar Chart - Year Vs Population</b>",
            xaxis=dict(
                title="<b>Year</b>",
                zeroline=False,
                gridcolor="rgb(183,183,183)",
                showline=True,
            ),
            yaxis=dict(
                title="<b>Population</b>",
                gridcolor="rgb(183,183,183)",
                zeroline=False,
                showline=True,
            ),
            font=dict(family="Courier New, monospace", size=12, color="rgb(0,0,0)"),
        ),
    }
)


In [ ]:
import plotly.io as pio


pio.write_html(fig, file='index.html', auto_open=True)

In [15]:
html_str = (
    '<p><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16"><path fill-rule="evenodd" d="M1.5 8a6.5 6.5 0 1113 0 6.5 6.5 0 01-13 0zM8 0a8 8 0 100 16A8 8 0 008 0zm.5 4.75a.75.75 0 00-1.5 0v3.5a.75.75 0 00.471.696l2.5 1a.75.75 0 00.557-1.392L8.5 7.742V4.75z"></path></svg> Latest Data: '
    + data_updated
    + "</p>"
)
with open("_includes/update.html", "w") as file:
    file.write(html_str)